In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

from vatools.src import util
from vatools.src.db_conn import DB_Connection
from vatools.src import ml

In [3]:
db = DB_Connection('../credentials.json')
db.connect()

## Exploring the Data

In [4]:
outcomes = db.query('select * from outcomes;')
projects = db.query('select * from projects;')

In [5]:
db.profile('outcomes')

Number of variables,12
Number of observations,353151
Total Missing (%),9.0%
Total size in memory,32.3 MiB
Average record size in memory,96.0 B
Numeric,3
Categorical,8
Boolean,0
Date,0
Text (Unique),1
Rejected,0


In [6]:
db.profile('projects')

Number of variables,35
Number of observations,353151
Total Missing (%),2.2%
Total size in memory,94.3 MiB
Average record size in memory,280.0 B
Numeric,5
Categorical,29
Boolean,0
Date,0
Text (Unique),1
Rejected,0


Below is a brief explanation of the provided data fields. Descriptions of self-explanatory names are omitted.

outcomes.csv  
**is_exciting** - ground truth of whether a project is exciting from business perspective  
**at_least_1_teacher_referred_donor** - teacher referred = donor donated because teacher shared a link or publicized their page  
**fully_funded** - project was successfully completed  
**at_least_1_green_donation** - a green donation is a donation made with credit card, PayPal, Amazon or check  
**great_chat** - project has a comment thread with greater than average unique comments  
**three_or_more_non_teacher_referred_donors** - non-teacher referred is a donor that landed on the site by means other than a teacher referral link/page  
**one_non_teacher_referred_donor_giving_100_plus** - see above  
**donation_from_thoughtful_donor** - a curated list of ~15 donors that are power donors and picky choosers (we trust them selecting great projects)  
**great_messages_proportion** -  how great_chat is calculated. proportion of comments on the project page that are unique. If > avg (currently 62%) then great_chat = True  
**teacher_referred_count** - number of donors that were teacher referred (see above)  
**non_teacher_referred_count** - number of donors that were non-teacher referred (see above)  

projects.csv
**projectid** - project's unique identifier  
**teacher_acctid** - teacher's unique identifier (teacher that created a project)  
**schoolid** - school's unique identifier (school where teacher works)  
**school_ncesid** - public National Center for Ed Statistics id  
**school_latitude**  
**school_longitude**  
**school_city**  
**school_state**  
**school_zip**  
**school_metro**  
**school_district**   
**school_county**  
**school_charter** - whether a public charter school or not (no private schools in the dataset)  
**school_magnet** - whether a public magnet school or not  
**school_year_round** - whether a public year round school or not  
**school_nlns** - whether a public nlns school or not  
**school_kipp** - whether a public kipp school or not  
**school_charter_ready_promise** - whether a public ready promise school or not  
**teacher_prefix** - teacher's gender  
**teacher_teach_for_america** - Teach for America or not  
teacher_ny_teaching_fellow** - New York teaching fellow or not  
**primary_focus_subject** - main subject for which project materials are intended  
**primary_focus_area** - main subject area for which project materials are intended  
**secondary_focus_subject** - secondary subject  
**secondary_focus_area** - secondary subject area  
**resource_type** - main type of resources requested by a project
poverty_level - school's poverty level.   
highest: 65%+ free of reduced lunch  
high: 40-64%  
moderate: 10-39%  
low: 0-9%  

**grade_level** - grade level for which project materials are intended  
**fulfillment_labor_materials** - cost of fulfillment  
**total_price_excluding_optional_support** - project cost excluding optional tip that donors give to DonorsChoose.org while funding a project  
**total_price_including_optional_support** - see above  
**students_reached** - number of students impacted by a project (if funded)  
**eligible_double_your_impact_match** - project was eligible for a 50% off offer by a corporate partner (logo appears on a project, like Starbucks or Disney)  
**eligible_almost_home_match** - project was eligible for a $100 boost offer by a corporate partner  
**date_posted** - data a project went live on the site  

## Exploratory Data Analysis

# Preprocessing

In [7]:
outcomes.head()

,projectid,is_exciting,at_least_1_teacher_referred_donor,fully_funded,at_least_1_green_donation,great_chat,three_or_more_non_teacher_referred_donors,one_non_teacher_referred_donor_giving_100_plus,donation_from_thoughtful_donor,great_messages_proportion,teacher_referred_count,non_teacher_referred_count
0,fffe0bb8af3b9cd93046b49653cc923a,t,t,t,t,t,t,t,f,100.0,1.0,5.0
1,fffdf9d286e715165b60674ac9d05c6c,f,f,t,t,t,t,t,f,66.0,0.0,5.0
2,fffb88aa251449f1ba0b31d5552426b0,f,f,t,t,t,t,t,f,66.0,0.0,13.0
3,fffb183a77300b3756b8c1dba0b6350f,f,f,t,f,f,f,t,f,0.0,0.0,1.0
4,fffaaa855c693496ffd4778ae1ce5985,f,f,t,f,f,f,t,f,0.0,0.0,1.0


## Preprocessing Notes

Initial notes below, further preprocessing to be done this weekend.  

projectid -  
is_exciting - fix bool  
at_least_1_teacher_referred_donor - fix bool; 46,246 missing values  
fully_funded - fix bool  
at_least_1_green_donation - fix bool; 46,246 missing values  
great_chat - fix bool  
three_or_more_non_teacher_referred_donors - fix bool; 46,246 missing values  
one_non_teacher_referred_donor_giving_100_plus - fix bool; 46,246 missing values  
donation_from_thoughtful_donor - fix bool; 46,246 missing values  
great_messages_proportion -  
teacher_referred_count -  
non_teacher_referred_count

In [8]:
projects.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,resource_type,poverty_level,grade_level,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted
0,62526d85d2a1818432d03d600969e99c,ebc7c90b6c92a069432e0714b8d93dfd,5aca9711ff0e4b37db48701f46f73036,171371005566,41.972419,-88.174597,Bartlett,IL,60103,suburban,...,Other,moderate poverty,Grades 3-5,30.00,444.36,522.78,7.0,f,f,2013-12-31
1,33d59ac771b80222ad63ef0f4ac47ade,de83b4c1f6428a15032c207c1d5e572a,d91a805b213bf74ae77b94e0de2b73ad,160153000269,43.501154,-112.056780,Idaho Falls,ID,83402,urban,...,Supplies,high poverty,Grades 3-5,30.00,233.24,274.40,30.0,f,f,2013-12-31
2,1a3aaeffc56dd2a421e37d8298024c0a,f4c9ed095b85458dcf858e25f203af00,9310d3eb447a4e46bc5fc31ed007ceac,330261000086,42.888244,-71.320224,Derry,NH,03038,suburban,...,Technology,moderate poverty,Grades 6-8,30.00,285.09,335.40,230.0,f,f,2013-12-31
3,33aa19ee4da4c5adf47d0dfb84fab5ef,17768031eb40de8d4497dbb54df48742,9ac70da58322783f82152eecc140a812,510324001378,37.476158,-77.488397,Richmond,VA,23224,urban,...,Other,highest poverty,Grades PreK-2,30.00,232.94,274.05,18.0,f,f,2013-12-31
4,e31c0ea8b68f404699dfb0d39e9bc99b,0f1bc5b4700fd33383be104442660178,cb9f688cf59e3ee22a087d616ca8f5d7,170993000805,41.952851,-87.650233,Chicago,IL,60613,urban,...,Supplies,highest poverty,Grades 6-8,30.00,513.41,604.01,70.0,t,f,2013-12-31


### Projects Preprocess Notes

Initial notes below, further preprocessing to be done this weekend.  

projectid - None  
teacher_acctid - None  
schoolid - None  
school_ncesid - None  
school_latitude - None  
school_longitude - None  
school_city - None  
school_state - None  
school_zip - None  
school_metro - None  
school_district -  
school_county -  
school_charter -  
school_magnet -  
school_year_round -  
school_nlns -  
school_kipp -  
school_charter_ready_promise
teacher_prefix -  
teacher_teach_for_america -  
teacher_ny_teaching_fellow -  
primary_focus_subject - map to fewer categories  
primary_focus_area - map to fewer categories  
secondary_focus_subject - map to fewer categories  
secondary_focus_area - map to fewer categories  
resource_type -  
poverty_level -  
grade_level -  
fulfillment_labor_materials -  
total_price_excluding_optional_support -  
total_price_including_optional_support -  
students_reached -  
eligible_double_your_impact_match - fix bool  
eligible_almost_home_match - fix bool  
date_posted -  

In [9]:
run ../preprocess

Missing values in outcomes data imputed with "f".
Missing values in projects data imputed. See preprocess.py for details.
Booleans in Outcomes set to 1 and 0.
Booleans in Projects set to 1 and 0.
non_teacher_referred_count 5
[ -0.304  60.8   121.6   182.4   243.2   304.   ]
1    352845
2       295
3         9
5         1
4         1
Name: non_teacher_referred_count, dtype: int64
teacher_referred_count 5
[ -0.151  30.2    60.4    90.6   120.8   151.   ]
1    352962
2       170
3        14
4         3
5         2
Name: teacher_referred_count, dtype: int64
Columns in outcomes successfully discretized.
students_reached [5, True]
[1.0000e+00 2.1000e+01 2.6000e+01 5.0000e+01 1.2000e+02 1.2143e+04]
3    76852
1    74277
2    69414
5    67269
4    65238
Name: students_reached, dtype: int64
total_price_excluding_optional_support [5, True]
[3.7030000e+01 2.5519000e+02 3.7482000e+02 4.6929000e+02 6.8137000e+02
 1.3972541e+05]
1    70676
4    70636
3    70634
5    70616
2    70589
Name: total_pric

In [10]:
outcomes.columns

Index(['projectid', 'is_exciting', 'at_least_1_teacher_referred_donor',
       'fully_funded', 'at_least_1_green_donation', 'great_chat',
       'three_or_more_non_teacher_referred_donors',
       'one_non_teacher_referred_donor_giving_100_plus',
       'donation_from_thoughtful_donor', 'great_messages_proportion',
       'teacher_referred_count', 'non_teacher_referred_count',
       'non_teacher_referred_count_descr', 'teacher_referred_count_descr'],
      dtype='object')

In [11]:
projects.columns

Index(['projectid', 'teacher_acctid', 'schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_city', 'school_state',
       'school_zip', 'school_district', 'school_charter', 'school_magnet',
       'school_year_round', 'school_nlns', 'school_kipp',
       'school_charter_ready_promise', 'teacher_prefix',
       'teacher_teach_for_america', 'teacher_ny_teaching_fellow',
       'primary_focus_subject', 'secondary_focus_subject',
       'secondary_focus_area', 'fulfillment_labor_materials',
       'total_price_excluding_optional_support',
       'total_price_including_optional_support', 'students_reached',
       'eligible_double_your_impact_match', 'eligible_almost_home_match',
       'date_posted', 'students_reached_descr',
       'total_price_excluding_optional_support_descr',
       'total_price_including_optional_support_descr',
       'grade_level_grades_3_5', 'grade_level_grades_6_8',
       'grade_level_grades_9_12', 'grade_level_grades_prek_2',
       

In [ ]:
db.create_table_from_df(outcomes,'outcomes_cleaned')

In [ ]:
# db.create_table_from_df(projects,'projects_cleaned', sep =',')
# Had to manually load data to database due to error
# extra data after last expected column
# CONTEXT:  COPY projects_cleaned, line 66351: "a3da9e50d570bbb1bb597bec77cca48d,8b210d09d18d434e440305b64bb3537e,e20cefa5a7929da97b849f4ab03bebad,,..."

In [ ]:
# db.query('''
# CREATE TABLE combined as
# SELECT a.*,
#        b.*
# FROM projects_cleaned a
# JOIN outcomes_cleaned b
# ON a.projectid = b.projectid_o''',False)

In [ ]:
combined_data = db.query('select * from combined')

In [ ]:
features = [
'school_charter',
'school_magnet',
'school_year_round',
'school_kipp',
'school_charter_ready_promise',
'teacher_teach_for_america',
'teacher_ny_teaching_fellow',
'eligible_double_your_impact_match',
'eligible_almost_home_match',
# 'students_reached_descr',
'total_price_excluding_optional_support_descr',
'total_price_including_optional_support_descr',
'grade_level_grades_3_5',
'grade_level_grades_6_8',
'grade_level_grades_9_12',
'grade_level_grades_prek_2',
'grade_level_missing',
'poverty_level_high_poverty',
'poverty_level_highest_poverty',
'poverty_level_low_poverty',
'poverty_level_moderate_poverty',
'primary_focus_area_applied_learning',
'primary_focus_area_health_and_sports',
'primary_focus_area_history_and_civics',
'primary_focus_area_literacy_and_language',
'primary_focus_area_math_and_science',
'primary_focus_area_missing',
'primary_focus_area_music_and_the_arts',
'primary_focus_area_special_needs',
'resource_type_books',
'resource_type_missing',
'resource_type_other',
'resource_type_supplies',
'resource_type_technology',
'resource_type_trips',
'resource_type_visitors',
'school_metro_missing',
'school_metro_rural',
'school_metro_suburban',
'school_metro_urban',
'school_county_cook',
'school_county_los_angeles',
'school_county_other']
outcome = 'fully_funded'
date_col = 'date_posted'

In [ ]:
# Changing positive case to not getting fully funded
combined_data['fully_funded'] = combined_data['fully_funded'].apply(lambda x: 1 if x == 0 else 0)

## Building Models

In [ ]:
temporal = {'start_date': '01/01/2011',
            'train_months': 12, 
            'test_months': 12, 
            'gap_months': 0,
            'periods': 2, 
            'increment_months': 12}
ml.mlpipeline(combined_data, features, outcome, methods = ['logit','dt','rf'], temporal = temporal, date_col = date_col)